In [1]:
import os
import json
from zipfile import ZipFile

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [3]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [4]:
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [7]:
data = pd.read_csv('IMDB Dataset.csv')
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [8]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [9]:
dataset = data.copy()

In [10]:
# Replacing words with 1 and 0 s
dataset.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)
dataset

C:\Users\HP\AppData\Local\Temp\ipykernel_13788\2912495443.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [11]:
dataset['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

Preparing Data for Preprocessing

In [12]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [13]:
train_data.shape, test_data.shape

((40000, 2), (10000, 2))

Data Preprocessing

In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen= 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen= 200)

In [15]:
print(X_train)
print(X_test)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [16]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

LSTM (Long Short-Term Memory) Model

In [17]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))  # First LSTM layer
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))  # Second LSTM layer
model.add(Dropout(0.5))  # Dropout layer with 50% dropout rate
model.add(Dense(64, activation='relu')) 
model.add(Dense(1, activation='sigmoid'))

c:\Users\HP\Desktop\Data Analytics + Science\Deep Learning Projects\Sentiment Analysis LSTM\myvenv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Training the Model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 396s 755ms/step - accuracy: 0.7083 - loss: 0.5462 - val_accuracy: 0.8010 - val_loss: 0.4383
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 360s 710ms/step - accuracy: 0.8254 - loss: 0.4022 - val_accuracy: 0.8652 - val_loss: 0.3339
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 380s 705ms/step - accuracy: 0.8508 - loss: 0.3593 - val_accuracy: 0.8414 - val_loss: 0.3772
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 416s 772ms/step - accuracy: 0.8506 - loss: 0.3565 - val_accuracy: 0.8680 - val_loss: 0.3181
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 538s 965ms/step - accuracy: 0.8823 - loss: 0.2932 - val_accuracy: 0.8681 - val_loss: 0.3227
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 517s 1s/step - accuracy: 0.8850 - loss: 0.2857 - val_accuracy: 0.8436 - val_loss: 0.3599
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 497s 902ms/step - accuracy: 0.8785 - loss: 0.3001 - val_accuracy: 0.8744 - val_loss: 0.3054
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 504s 905ms/step - accuracy: 0.9087 - lo

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 58s 177ms/step - accuracy: 0.8864 - loss: 0.2789


Building a Prediction System

In [21]:
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = 'positive' if prediction[0][0] > 0.5 else 'negative'
    return sentiment

In [22]:
input_review = "This Movie has a nice plot. I do like it"
sentiment = predict_sentiment(input_review)
print(f'The Sentiment of the Review is: {sentiment}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The Sentiment of the Review is: positive


In [23]:
model.save('Movie_Reviews_Model.h5') # or model.save('Movie_Reviews_Model.keras')

In [24]:
# Saving Tokenizer Configuration
tokenizer_config = tokenizer.to_json()
with open('tokenizer_config.json', 'w') as json_file:
    json.dump(tokenizer_config, json_file)

In [25]:
# Saving Word Index
with open('tokenizer_word_index.json', 'w') as json_file:
    json.dump(tokenizer.word_index, json_file)